# Load required modules

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Class to define the neural network

In [2]:
#Adapted from https://github.com/pytorch/examples/tree/main/mnist
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

# Neural network trainer function

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Main function - calls the trainer and save the trained model to file

In [4]:
def main():
    epochs = 1
    lr = 0.01
    gamma = 0.03
    use_cuda = 0
    save_model = 1
    device = torch.device("cpu")
    train_kwargs = {'batch_size': 100}
    test_kwargs = {'batch_size': 10}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)

    dataset_mnist = torch.load('train_dataset_mnist.pth')
    train_loader = torch.utils.data.DataLoader(dataset_mnist,**train_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        #test(model, device, test_loader)
        scheduler.step()

    if save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")

In [5]:
if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.283808
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.246888
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.187686
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.113077
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.018864
Train Epoch: 1 [5000/60000 (8%)]	Loss: 1.917313
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.631829
Train Epoch: 1 [7000/60000 (12%)]	Loss: 1.584600
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.403944
Train Epoch: 1 [9000/60000 (15%)]	Loss: 1.294302
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.209682
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.964514
Train Epoch: 1 [12000/60000 (20%)]	Loss: 1.028847
Train Epoch: 1 [13000/60000 (22%)]	Loss: 1.258839
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.916509
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.722350
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.127907
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.935521
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.845479
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.861990
Train Epoch: 1 [200